# Lag 생성

In [ ]:
# processed_data/renamed_csv에 위치한 csv 파일들로 Lag 데이터 프레임 생성
from pathlib import Path
import pandas as pd
from functools import reduce

# processed_data에 위치한 csv 파일들로 Lag 데이터 프레임 생성
input_dir = Path("processed_data")
csv_files = list(input_dir.glob("*.csv"))


In [ ]:
# 각 csv 파일을 읽어와서 데이터프레임으로 변환
# 읽어오는 파일 이름명에서 연도정보를 추출하여 "year" 컬럼으로 추가
dfs = []
for file in csv_files:
    year = file.stem.split('_')[2]  # 파일 이름에서 연도 추출
    df = pd.read_csv(file, thousands=',')  # 천 단위 구분자 처리
    df['year'] = int(year)  # year 컬럼 추가 
    dfs.append(df)

In [ ]:
# 예: "의창구"값을 포함하는 행을 찾으면, "성산구"값을 포함하는 행을 찾을때까지 읽은 행은 모두 "의창구" 데이터프레임에 포함.
# 이후 "성산구"값을 포함하는 행을 찾으면, "마산합포구"값을 포함하는 행을 찾을때까지 읽은 행은 모두 "성산구" 데이터프레임에 포함.
# 이와 같은 작업을 반복하여 5개의 데이터프레임 생성. "진해구"는 마지막 행까지 포함.
# 마커(구 이름) 행을 기준으로 DataFrame을 5개 구간으로 분할하는 함수
markers = ["의창구", "성산구", "마산합포구", "마산회원구", "진해구"]

def split_by_markers(df, col="행정동", markers=markers):
    s = df[col].astype(str).str.strip()
    starts = []
    for m in markers:
        idx_list = df.index[s.eq(m)].tolist()
        if not idx_list:
            raise ValueError(f"마커 '{m}'를 찾을 수 없습니다.")
        starts.append(idx_list[0])

    # 마커가 파일 내에서 올바른 순서(오름차순)인지 검증
    if any(starts[i] >= starts[i+1] for i in range(len(starts)-1)):
        raise ValueError(f"마커 인덱스 순서가 올바르지 않습니다: {starts}")

    parts = {}
    for i, m in enumerate(markers[:-1]):
        parts[m] = df.iloc[starts[i]:starts[i+1]].reset_index(drop=True).copy()
    parts[markers[-1]] = df.iloc[starts[-1]:].reset_index(drop=True).copy()
    return parts

# 모든 연도 CSV(dfs 리스트)에 대해 분할 후, 구별로 합치기
buckets = {m: [] for m in markers}
for one_df in dfs:
    split = split_by_markers(one_df, col="행정동", markers=markers)
    for m in markers:
        buckets[m].append(split[m])

# 최종 5개 데이터프레임
uichang_df     = pd.concat(buckets["의창구"], ignore_index=True)
seongsan_df    = pd.concat(buckets["성산구"], ignore_index=True)
masanhappo_df  = pd.concat(buckets["마산합포구"], ignore_index=True)
masanhwewon_df = pd.concat(buckets["마산회원구"], ignore_index=True)
jinhae_df      = pd.concat(buckets["진해구"], ignore_index=True)

# 각 데이터프레임에 대해 행정동으로 그룹화 후 year 순으로 정렬 (inplace 사용)
for df in [uichang_df, seongsan_df, masanhappo_df, masanhwewon_df, jinhae_df]:
    df["행정동"] = df["행정동"].astype(str).str.strip()
    df["년월"] = pd.to_datetime(df["year"].astype(str), format="%Y%m", errors="coerce")
    df.sort_values(["행정동", "년월"], kind="mergesort", inplace=True)
    df.reset_index(drop=True, inplace=True)
    df["year"] = df["년월"].dt.strftime("%Y%m").astype(int)
    df.drop(columns=["년월"], inplace=True)

# 확인
for name, d in [("의창구", uichang_df), ("성산구", seongsan_df),
                ("마산합포구", masanhappo_df), ("마산회원구", masanhwewon_df),
                ("진해구", jinhae_df)]:
    print(name, len(d))

In [ ]:
# 5개 데이터프레임에 대해 누락값 처리
for df in [uichang_df, seongsan_df, masanhappo_df, masanhwewon_df, jinhae_df]:
    df.dropna(subset=["소계"], inplace=True)

In [ ]:
# 최종적으로 추이를 추적할 행정동 리스트는 다음과 같음.
"""
의창구 : 동읍,북면,대산면,의창동,팔룡동,명곡동,봉림동
성산구 : 반송동,용지동,중앙동,상남동,사파동,가음정동,성주동,웅남동
마산합포구 : 구산면,진동면,진북면,진전면,현동,가포동,월영동,문화동,반월중앙동,완월동,자산동,교방동,오동동,합포동,산호동
마산회원구 : 내서읍,회원1동,회원2동,석전동,회성동,양덕1동,양덕2동,합성1동,합성2동,구암1동,구암2동,봉암동
진해구 : 충무동,여좌동,태백동,경화동,병암동,석동,이동,자은동,덕산동,풍호동,웅천동,웅동1동,웅동2동
"""
# lag 데이터 생성은 다음 유의사항을 참고하여 생성 할 것임.
# 행정동 편입에 따른 이상치 ( 의창구 용지동 -> 성산구 용지동 ) 변화에 대응하기위해 편입 당월 데이터의 흐름을 고려하여 데이터 프레임 재편.
"""
의창구 동읍, 북면, 대산면, 의창동, 팔룡동, 명곡동, 봉림동은 ui-chang_df를 그대로 써도 됨.
성산구 반송동, 상남동, 사파동, 가음정동, 성주동, 웅남동은 seongsan_df를 그대로 써도 됨.
마산합포구 구산면, 진동면, 진북면, 진전면, 현동, 가포동, 월영동, 문화동, 완월동, 자산동,
            합포동, 산호동은 masanhappo_df를 그대로 써도 됨.
마산회원구 내서읍, 회원1동, 회원2동, 회성동, 양덕1동, 양덕2동, 합성1동, 합성2동, 구암1동,
            구암2동, 봉암동은 masanhwewon_df를 그대로 써도 됨.
진해구 여좌동, 태백동, 경화동, 병암동, 석동, 이동, 자은동, 덕산동, 풍호동, 웅천동, 웅동1동,
            웅동2동은 jinhae_df를 그대로 써도 됨.

우선 상기 행정동에 대한 1-step lag 데이터를 생성.
"""
# Lag 생성
lag_months = 1
for df in [uichang_df, seongsan_df, masanhappo_df, masanhwewon_df, jinhae_df]:
    df.sort_values(by=['행정동', 'year'], inplace=True)
    df.reset_index(drop=True, inplace=True)
    df[f'소계_Lag{lag_months}'] = df.groupby('행정동')['소계'].shift(lag_months)
    # 전년도 대비 변화값 컬럼 추가. (예: 202101 - 202001)
    df[f'소계_YoY_Change'] = df['소계'] - df['소계_Lag1']


# 편입한 행정동 데이터 개별 대응

## 성산구 중앙동

In [ ]:
# 행정구역 조정('의창구 대원동·두대동·삼동동·덕정동(법정동) →  성산구 중앙동)에 따른 이상치 발생.
# 그외 특이사항없으므로, seongsan_df의 중앙동 데이터는 해당 202107월 행만 보정하고 나머지는 그대로 사용함.
# 필요에 따라 다른 코드블럭에서 보정할것.

## 성산구 용지동

In [ ]:
# 이중, "성산구 용지동"은 2021년 7월에 "의창구 용지동"에서 "성산구 용지동"으로 편입되었으므로,
# 의창구 데이터프레임에서는 21.06~21.07 구간의 정보를 사용하고, 
# 성산구 데이터프레임에서는 21.07~25.08 구간의 정보를 사용하는것이 타당함.
# 두 데이터프레임의 정보를 결합하여 특별 데이터프레임을 생성
seongsan_youngji_df = pd.concat([uichang_df, seongsan_df], axis=0)
seongsan_youngji_df = seongsan_youngji_df[seongsan_youngji_df['행정동'] == '용지동'].copy()
# 수동으로 202107의 "소계_Lag1" 값을 202106의 "소계" 값으로 채움
seongsan_youngji_df.loc[seongsan_youngji_df['year'] == 202107, '소계_Lag1'] = seongsan_youngji_df.loc[seongsan_youngji_df['year'] == 202106, '소계'].values
# 전년도 대비 변화값 컬럼 재계산
seongsan_youngji_df.loc[seongsan_youngji_df['year'] == 202107, '소계_YoY_Change'] = seongsan_youngji_df.loc[seongsan_youngji_df['year'] == 202107, '소계'] - seongsan_youngji_df.loc[seongsan_youngji_df['year'] == 202107, '소계_Lag1']
seongsan_youngji_df.sort_values(by=['year'], inplace=True)
seongsan_youngji_df.reset_index(drop=True, inplace=True)
# 최종 프레임을 CSV로 저장(필요시)
seongsan_youngji_df.to_csv("case/resident_population_seongsan_youngji.csv", index=False, encoding='utf-8-sig')


## 마산합포구 반월중앙동

In [ ]:
# "마산합포구 반월중앙동"은 201701에 "마산합포구 반월동"과 "마산합포구 중앙동"이 통합.
# 따라서, 201701 이전의 데이터는 "반월동"과 "중앙동"의 데이터를 합산하여 사용.
# 201701 이후의 데이터는 "반월중앙동" 데이터를 사용.
# 두 데이터프레임의 정보를 결합하여 특별 데이터프레임을 생성
masanhappo_banwol_df = pd.concat([masanhappo_df], axis=0)
banwol = masanhappo_banwol_df[masanhappo_banwol_df['행정동'] == '반월동'].copy()
jungang = masanhappo_banwol_df[masanhappo_banwol_df['행정동'] == '중앙동'].copy()
banwol = banwol.drop(columns=['행정동'])
jungang = jungang.drop(columns=['행정동'])
# 201701 이전 데이터 합산
# 두 데이터프레임의 행을 같은 year값을 가진 행끼리 데이터값을 합연산으로 수행해서 새로운 데이터프레임 생성
banwol_jungang_combined = pd.merge(banwol, jungang, on='year', suffixes=('_banwol', '_jungang'))
banwol_jungang_combined['행정동'] = '반월중앙동'
banwol_jungang_combined['소계'] = banwol_jungang_combined['소계_banwol'] + banwol_jungang_combined['소계_jungang']
banwol_jungang_combined = banwol_jungang_combined[['행정동', '소계', 'year']]
# 201701 이후 데이터
banwol_jungang_after_201701 = masanhappo_banwol_df[(masanhappo_banwol_df['행정동'] == '반월중앙동') & (masanhappo_banwol_df['year'] >= 201701)].copy()
# 두 데이터프레임 결합
masanhappo_banwol_df = pd.concat([banwol_jungang_combined[banwol_jungang_combined['year'] < 201701], banwol_jungang_after_201701], ignore_index=True)
masanhappo_banwol_df.sort_values(by=['year'], inplace=True)
masanhappo_banwol_df.reset_index(drop=True, inplace=True)
# Lag1, YoY_Change 컬럼 재생성
masanhappo_banwol_df[f'소계_Lag{lag_months}'] = masanhappo_banwol_df['소계'].shift(lag_months)  
masanhappo_banwol_df[f'소계_YoY_Change'] = masanhappo_banwol_df['소계'] - masanhappo_banwol_df[f'소계_Lag{lag_months}']
masanhappo_banwol_df = masanhappo_banwol_df[['행정동', '소계', 'year', f'소계_Lag{lag_months}', '소계_YoY_Change']]
# 최종 프레임을 CSV로 저장(필요시)
masanhappo_banwol_df.to_csv("case/resident_population_masanhappo_banwol.csv", index=False, encoding='utf-8-sig')


## 마산합포구 교방동

In [ ]:
# 202001에 "마산합포구 교방동"과 "마산합포구 노산동"이 통합되어 "마산합포구 교방동"이 되었음.
# 따라서, 202001 이전의 데이터는 "교방동"과 "노산동"의 데이터를 합산하여 사용.
# 202001 이후의 데이터는 "교방동" 데이터를 사용.
# 두 데이터프레임의 정보를 결합하여 특별 데이터프레임을 생성
masanhappo_gyobang_df = pd.concat([masanhappo_df], axis=0)
gyobang = masanhappo_gyobang_df[masanhappo_gyobang_df['행정동'] == '교방동'].copy()
nosan = masanhappo_gyobang_df[masanhappo_gyobang_df['행정동'] == '노산동'].copy()
gyobang = gyobang.drop(columns=['행정동'])
nosan = nosan.drop(columns=['행정동'])
# 202001 이전 데이터 합산
# 두 데이터프레임의 행을 같은 year값을 가진 행끼리 데이터값을 합연산으로 수행해서 새로운 데이터프레임 생성
gyobang_nosan_combined = pd.merge(gyobang, nosan, on='year', suffixes=('_gyobang', '_nosan'))
gyobang_nosan_combined['행정동'] = '교방동'
gyobang_nosan_combined['소계'] = gyobang_nosan_combined['소계_gyobang'] + gyobang_nosan_combined['소계_nosan']
gyobang_nosan_combined = gyobang_nosan_combined[['행정동', '소계', 'year']]
# 202001 이후 데이터
gyobang_nosan_after_202001 = masanhappo_gyobang_df[(masanhappo_gyobang_df['행정동'] == '교방동') & (masanhappo_gyobang_df['year'] >= 202001)].copy()
# 두 데이터프레임 결합
masanhappo_gyobang_df = pd.concat([gyobang_nosan_combined[gyobang_nosan_combined['year'] < 202001], gyobang_nosan_after_202001], ignore_index=True)
masanhappo_gyobang_df.sort_values(by=['year'], inplace=True)
masanhappo_gyobang_df.reset_index(drop=True, inplace=True)
# Lag1, YoY_Change 컬럼 재생성
masanhappo_gyobang_df[f'소계_Lag{lag_months}'] = masanhappo_gyobang_df['소계'].shift(lag_months)  
masanhappo_gyobang_df[f'소계_YoY_Change'] = masanhappo_gyobang_df['소계'] - masanhappo_gyobang_df[f'소계_Lag{lag_months}']
masanhappo_gyobang_df = masanhappo_gyobang_df[['행정동', '소계', 'year', f'소계_Lag{lag_months}', '소계_YoY_Change']]
# 최종 데이터프레임을 CSV로 저장 (필요시)
masanhappo_gyobang_df.to_csv('case/masanhappo_gyobang_df.csv', index=False)


## 마산합포구 오동동

In [ ]:
# 201701에 "마산합포구 동서동", "마산합포구 성호동, "마산합포구 오동동"이 통합되어 "마산합포구 오동동"이 되었음.
# 따라서, 201701 이전의 데이터는 "동서동"과 "성호동", "오동동"의 데이터를 합산하여 사용.
# 201701 이후의 데이터는 "오동동" 데이터를 사용.
# 두 데이터프레임의 정보를 결합하여 특별 데이터프레임을 생성
masanhappo_odong_df = pd.concat([masanhappo_df], axis=0)
dongseo = masanhappo_odong_df[masanhappo_odong_df['행정동'] == '동서동'].copy()
seongho = masanhappo_odong_df[masanhappo_odong_df['행정동'] == '성호동'].copy()
odong = masanhappo_odong_df[masanhappo_odong_df['행정동'] == '오동동'].copy()
dongseo = dongseo.drop(columns=['행정동'])
seongho = seongho.drop(columns=['행정동'])
odong = odong.drop(columns=['행정동'])
# 201701 이전 데이터 합산
# 세 데이터프레임의 행을 같은 year값을 가진 행끼리 데이터값을 합연산으로 수행해서 새로운 데이터프레임 생성
dongseo_seongho_odong_combined = reduce(lambda left, right: pd.merge(left, right, on='year', suffixes=('_left', '_right')), [dongseo, seongho, odong])
dongseo_seongho_odong_combined['행정동'] = '오동동'
dongseo_seongho_odong_combined['소계'] = (dongseo_seongho_odong_combined['소계_left'] + 
                                   dongseo_seongho_odong_combined['소계_right'] + 
                                   dongseo_seongho_odong_combined['소계'])
dongseo_seongho_odong_combined = dongseo_seongho_odong_combined[['행정동', '소계', 'year']]
# 201701 이후 데이터
dongseo_seongho_odong_after_201701 = masanhappo_odong_df[(masanhappo_odong_df['행정동'] == '오동동') & (masanhappo_odong_df['year'] >= 201701)].copy()
# 두 데이터프레임 결합
masanhappo_odong_df = pd.concat([dongseo_seongho_odong_combined[dongseo_seongho_odong_combined['year'] < 201701], dongseo_seongho_odong_after_201701], ignore_index=True)
masanhappo_odong_df.sort_values(by=['year'], inplace=True)
masanhappo_odong_df.reset_index(drop=True, inplace=True)
# Lag1, YoY_Change 컬럼 재생성
masanhappo_odong_df[f'소계_Lag{lag_months}'] = masanhappo_odong_df['소계'].shift(lag_months)  
masanhappo_odong_df[f'소계_YoY_Change'] = masanhappo_odong_df['소계'] - masanhappo_odong_df[f'소계_Lag{lag_months}']
masanhappo_odong_df = masanhappo_odong_df[['행정동', '소계', 'year', f'소계_Lag{lag_months}', '소계_YoY_Change']]
# 최종 데이터프레임을 CSV로 저장 (필요시)
masanhappo_odong_df.to_csv('case/masanhappo_odong_df.csv', index=False)

## 마산회원구 석전동

In [ ]:
# 201701에 "마산회원구 석전1동", "마산회원구 석전2동"이 통합되어 "마산회원구 석전동"이 되었음.
# 따라서, 201701 이전의 데이터는 "석전1동"과 "석전2동"의 데이터를 합산하여 사용.
# 201701 이후의 데이터는 "석전동" 데이터를 사용.
# 두 데이터프레임의 정보를 결합하여 특별 데이터프레임을 생성
masanhwewon_seokjeon_df = pd.concat([masanhwewon_df], axis=0)
seokjeon1 = masanhwewon_seokjeon_df[masanhwewon_seokjeon_df['행정동'] == '석전1동'].copy()
seokjeon2 = masanhwewon_seokjeon_df[masanhwewon_seokjeon_df['행정동'] == '석전2동'].copy()
seokjeon1 = seokjeon1.drop(columns=['행정동'])
seokjeon2 = seokjeon2.drop(columns=['행정동'])
# 201701 이전 데이터 합산
# 두 데이터프레임의 행을 같은 year값을 가진 행끼리 데이터값을 합연산으로 수행해서 새로운 데이터프레임 생성
seokjeon1_seokjeon2_combined = pd.merge(seokjeon1, seokjeon2, on='year', suffixes=('_seokjeon1', '_seokjeon2'))
seokjeon1_seokjeon2_combined['행정동'] = '석전동'
seokjeon1_seokjeon2_combined['소계'] = seokjeon1_seokjeon2_combined['소계_seokjeon1'] + seokjeon1_seokjeon2_combined['소계_seokjeon2']
seokjeon1_seokjeon2_combined = seokjeon1_seokjeon2_combined[['행정동', '소계', 'year']]
# 201701 이후 데이터
seokjeon1_seokjeon2_after_201701 = masanhwewon_seokjeon_df[(masanhwewon_seokjeon_df['행정동'] == '석전동') & (masanhwewon_seokjeon_df['year'] >= 201701)].copy()
# 두 데이터프레임 결합
masanhwewon_seokjeon_df = pd.concat([seokjeon1_seokjeon2_combined[seokjeon1_seokjeon2_combined['year'] < 201701], seokjeon1_seokjeon2_after_201701], ignore_index=True)
masanhwewon_seokjeon_df.sort_values(by=['year'], inplace=True)
masanhwewon_seokjeon_df.reset_index(drop=True, inplace=True)
# Lag1, YoY_Change 컬럼 재생성
masanhwewon_seokjeon_df[f'소계_Lag{lag_months}'] = masanhwewon_seokjeon_df['소계'].shift(lag_months)  
masanhwewon_seokjeon_df[f'소계_YoY_Change'] = masanhwewon_seokjeon_df['소계'] - masanhwewon_seokjeon_df[f'소계_Lag{lag_months}']
masanhwewon_seokjeon_df = masanhwewon_seokjeon_df[['행정동', '소계', 'year', f'소계_Lag{lag_months}', '소계_YoY_Change']]
# 최종 데이터프레임을 CSV로 저장 (필요시)
masanhwewon_seokjeon_df.to_csv('case/masanhwewon_seokjeon_df.csv', index=False)

## 진해구 충무동

In [ ]:
# 202001에 "진해구 중앙동", "진해구 태평동", "진해구 충무동"이 통합되어 "진해구 충무동"이 되었음.
# 따라서, 202001 이전의 데이터는 "중앙동", "태평동", "충무동"의 데이터를 합산하여 사용.
# 202001 이후의 데이터는 "충무동" 데이터를 사용.
# 두 데이터프레임의 정보를 결합하여 특별 데이터프레임을 생성
jinhae_chungmu_df = pd.concat([jinhae_df], axis=0)
chungmu = jinhae_chungmu_df[jinhae_chungmu_df['행정동'] == '충무동'].copy()
jungang = jinhae_chungmu_df[jinhae_chungmu_df['행정동'] == '중앙동'].copy()
taepyeong = jinhae_chungmu_df[jinhae_chungmu_df['행정동'] == '태평동'].copy()
chungmu = chungmu.drop(columns=['행정동'])
jungang = jungang.drop(columns=['행정동'])
taepyeong = taepyeong.drop(columns=['행정동'])
# 202001 이전 데이터 합산
# 세 데이터프레임의 행을 같은 year값을 가진 행끼리 데이터값을 합연산으로 수행해서 새로운 데이터프레임 생성
jungang_taepyeong_chungmu_combined = reduce(lambda left, right: pd.merge(left, right, on='year', suffixes=('_left', '_right')), [jungang, taepyeong, chungmu])
jungang_taepyeong_chungmu_combined['행정동'] = '충무동'
jungang_taepyeong_chungmu_combined['소계'] = (jungang_taepyeong_chungmu_combined['소계_left'] + 
                                       jungang_taepyeong_chungmu_combined['소계_right'] + 
                                       jungang_taepyeong_chungmu_combined['소계'])
jungang_taepyeong_chungmu_combined = jungang_taepyeong_chungmu_combined[['행정동', '소계', 'year']]
# 202001 이후 데이터
jungang_taepyeong_chungmu_after_202001 = jinhae_chungmu_df[(jinhae_chungmu_df['행정동'] == '충무동') & (jinhae_chungmu_df['year'] >= 202001)].copy()
# 두 데이터프레임 결합
jinhae_chungmu_df = pd.concat([jungang_taepyeong_chungmu_combined[jungang_taepyeong_chungmu_combined['year'] < 202001], jungang_taepyeong_chungmu_after_202001], ignore_index=True)
jinhae_chungmu_df.sort_values(by=['year'], inplace=True)
jinhae_chungmu_df.reset_index(drop=True, inplace=True)
# Lag1, YoY_Change 컬럼 재생성
jinhae_chungmu_df[f'소계_Lag{lag_months}'] = jinhae_chungmu_df['소계'].shift(lag_months)  
jinhae_chungmu_df[f'소계_YoY_Change'] = jinhae_chungmu_df['소계'] - jinhae_chungmu_df[f'소계_Lag{lag_months}']
jinhae_chungmu_df = jinhae_chungmu_df[['행정동', '소계', 'year', f'소계_Lag{lag_months}', '소계_YoY_Change']]
# 최종 데이터프레임을 CSV로 저장 (필요시)
jinhae_chungmu_df.to_csv('case/jinhae_chungmu_df.csv', index=False)

In [ ]:
# 최종 데이터프레임을 CSV로 저장
for name, df in [("의창구", uichang_df), ("성산구", seongsan_df),
                ("마산합포구", masanhappo_df), ("마산회원구", masanhwewon_df),
                ("진해구", jinhae_df)]:
    output_file = input_dir / f"{name}_Lag_Processed.csv"
    df.to_csv(output_file, index=False, encoding='utf-8-sig')
    print(f"Saved {output_file}")